In [1]:
import numpy as np 
import pandas as pd 
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 
import os
os.chdir('C:/Users/Neil/Google Drive/Public Health/Code/Covid-19')

## from https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset

## tutorial and introduction to plotly from Terence Shin, published in towardsdatascience

import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
dfconfirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv',
                index_col=[1,0])
df = dfconfirmed.drop(['Lat','Long'],axis=1).stack()

dfdeaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv',
           index_col=[1,0])
df1 = dfdeaths.drop(['Lat','Long'],axis=1).stack()

dfrecovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv',
           index_col=[1,0])
df2 = dfrecovered.drop(['Lat','Long'],axis=1).stack()

dfTimeSeries = pd.DataFrame([df,df1,df2],index=['Cases','Deaths','Recovered']).transpose().reorder_levels([2,0,1],axis=0)
dfTimeSeries.index = dfTimeSeries.index.set_levels([pd.to_datetime(dfTimeSeries.index.levels[0]),
                               dfTimeSeries.index.levels[1],
                               dfTimeSeries.index.levels[2]])
dfTimeSeries.sort_index(level=0,inplace=True)

In [3]:
today = dfTimeSeries[dfTimeSeries.index.get_level_values(0).max()::].reset_index().drop('level_0',axis=1)
countries_today = today.groupby('Country/Region').sum().reset_index()
countries_today['Current Cases'] = countries_today['Cases'] - countries_today[['Country/Region','Deaths','Recovered']].sum(axis=1)

## interactive colour maps

In [4]:
def chloropleth_frequencies(df,case_type,title):
    fig = go.Figure(data=go.Choropleth(
        locations = df['Country/Region'],
        locationmode = 'country names',
        z = df[case_type],
        #colorscale = 'Viridis',
         #autocolorscale=False,
        #reversescale=True,
        marker_line_color = 'black',
        marker_line_width = 0.5,
    ))

    fig.update_layout(
        title_text = title,
        title_x = 0.5,
        geo=dict(
            showframe = False,
            showcoastlines = False,
            projection_type = 'equirectangular'
        )
    )
    fig.show()

### Data from yesterday

In [5]:
countries_no_china = countries_today[countries_today['Country/Region']!='China']
chloropleth_frequencies(countries_today,'Current Cases','Current confirmed cases')
chloropleth_frequencies(countries_today,'Cases','Cumulative confirmed cases')
chloropleth_frequencies(countries_today,'Recovered','Cumulative recovered cases')
chloropleth_frequencies(countries_no_china,'Current Cases','Current confirmed cases excluding China')
chloropleth_frequencies(countries_no_china,'Cases','Cumulative confirmed cases excluding China')
chloropleth_frequencies(countries_no_china,'Recovered','Cumulative recovered cases excluding China')

In [6]:
countries_today['Current Cases'].sum()
#note that more people have recovered in China than are currently confirmed cases in the rest of the world

55299

In [7]:
countries_today[countries_today['Country/Region']=='China']

,Country/Region,Cases,Deaths,Recovered,Current Cases
24,China,80932,3172,62901,14859


In [8]:
df_countrydate = dfTimeSeries[dfTimeSeries['Cases']>0]
df_countrydate = df_countrydate.reset_index().groupby(['level_0','Country/Region']).sum().reset_index()
df_countrydate = df_countrydate.sort_values(by='level_0') # sort rows based on timestamp

df_countrydate['level_0'] = df_countrydate['level_0'].apply(lambda x: x.date()).apply(str) # convert timestamp to a string

df_countrydate_chinaless = df_countrydate[df_countrydate['Country/Region']!='China']
df_countrydate['Current Cases']=df_countrydate['Cases']-df_countrydate[['Deaths','Recovered']].sum(axis=1)
df_countrydate_chinaless['Current Cases']=df_countrydate_chinaless['Cases']-df_countrydate_chinaless[['Deaths','Recovered']].sum(axis=1)

C:\Users\Neil\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [9]:
fig = px.choropleth(df_countrydate, 
                    locations="Country/Region", 
                    locationmode = "country names",
                    color="Cases", 
                    hover_name="Country/Region", 
                    animation_frame="level_0"
                   )

fig.update_layout(
    title_text = 'Spread of Coronavirus',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig.show()

fig = px.choropleth(df_countrydate_chinaless, 
                    locations="Country/Region", 
                    locationmode = "country names",
                    color="Cases", 
                    hover_name="Country/Region", 
                    animation_frame="level_0"
                   )

fig.update_layout(
    title_text = 'Spread of Coronavirus',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig.show()

In [10]:
#outside of china at this point, the cruise ship Diamond Princess has the highest caseload
df_countrydate[df_countrydate['level_0'] == '2020-02-19']

,level_0,Country/Region,Cases,Deaths,Recovered,Current Cases
644,2020-02-19,Sri Lanka,1,0,1,0
640,2020-02-19,Philippines,3,1,1,1
641,2020-02-19,Russia,2,0,2,0
642,2020-02-19,Singapore,84,0,34,50
643,2020-02-19,Spain,2,0,2,0
645,2020-02-19,Sweden,1,0,0,1
651,2020-02-19,Vietnam,16,0,7,9
647,2020-02-19,Thailand,35,0,15,20
648,2020-02-19,US,13,0,3,10
649,2020-02-19,United Arab Emirates,9,0,4,5


In [11]:
fig = px.pie(countries_today, values = 'Current Cases',names='Country/Region', height=600)
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))

fig.show()

## Stacked barcharts comparing Total and Current Cases, and Deaths and Recoveries day-by-day by country

In [12]:
def plotly_bar(df,case_type):
    fig = px.bar(df, x="level_0", y=case_type, color='Country/Region', text = case_type, orientation='v', height=600,
             title=case_type)
    fig.show()
    
casetypes= ['Cases','Deaths','Recovered','Current Cases']

for i in casetypes:
    plotly_bar(df_countrydate,i)

# Growth in Countries with over 100 cases

In [13]:
countries_over_100 = countries_today.loc[countries_today['Cases']>100,'Country/Region']
df_over_100 = dfTimeSeries.loc[(slice(None),countries_over_100,slice(None)),:].reset_index().groupby(['level_0','Country/Region']).sum()
df5 = df_over_100.reset_index().pivot(index='level_0',columns='Country/Region',values='Cases')
new_cases = df5 - df5.shift(1).fillna(0)
new_cases = new_cases.stack().reset_index()
new_cases_chinaless = new_cases[new_cases['Country/Region']!='China']

In [14]:
## removed 12/03 from these data as many countries appear to have reported no new cases - likely not updated sitrep at this time

fig = px.line(new_cases[new_cases['level_0']!=new_cases['level_0'].max()], x="level_0", y=0, color='Country/Region', 
              title='Daily New Cases Confirmations by country')
fig.show()


In [21]:
df_over_100.loc[(slice(None),'Japan'),:]
#looks like there's a typo in Japan data for Feb 6, should be 25 cases which is why there is '-20' cases on feb 7

,,Cases,Deaths,Recovered
level_0,Country/Region,,,
2020-01-22,Japan,2,0,0
2020-01-23,Japan,1,0,0
2020-01-24,Japan,2,0,0
2020-01-25,Japan,2,0,0
2020-01-26,Japan,4,0,1
2020-01-27,Japan,4,0,1
2020-01-28,Japan,7,0,1
2020-01-29,Japan,7,0,1
2020-01-30,Japan,11,0,1


## Growth in cases after 100 days in natural figures and log of growth

In [76]:
df6 = df5[df5>100].apply(lambda x: pd.Series(x.dropna().values))
after_100 = df6 - df6.shift(1)#.stack()
new_cases_days_after_100 = after_100.drop('Cruise Ship',axis=1).dropna(how='all').replace(0,np.NaN).stack().reset_index()
new_cases_days_after_100['Log of New Cases'] = np.log(new_cases_days_after_100[0])
new_cases_days_after_100.columns

Index(['level_0', 'Country/Region', 0, 'Log of New Cases'], dtype='object')

In [77]:
fig = px.line(new_cases_days_after_100, x="level_0", y=0, color='Country/Region', 
              title='New Case Confirmations by Days after 100 Case Confirmations Reached')
fig.show()
#China apparently declared 15000 cases in one day which slightly messes up the scale

fig = px.line(new_cases_days_after_100, x="level_0", y='Log of New Cases', color='Country/Region', 
              title='New Case Confirmations by Days after 100 Case Confirmations Reached')
fig.show()
